## Anthropic Agent Class

In [2]:
from enum import Enum, auto
from anthropic import Anthropic
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional
import os
from dynamic_agent.utils.function_schema import create_function_schema, SchemaFormat
from smart_team.agents.agent_functions import (
    create_virtualenv,
    install_package,
    execute_code,
    get_weather,
)
class AgentState(Enum):
    """States that an agent can be in"""
    READY = auto()           # Ready to receive new input
    PROCESSING = auto()      # Processing a task
    COMPLETED = auto()       # Task completed, ready to transfer
    TRANSFERRING = auto()    # In process of transferring


In [3]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class AgentResponse:
    """Response from an agent"""
    text: str | None = None
    function_calls: list[dict[str, any]] | None = None


class BaseAgent(ABC):
    """Base class for all agents"""

    def __init__(self, name: str, instructions: str, functions: List = None, **kwargs):
        """Initialize the agent"""
        self.is_ochestrator: bool = False
        self.name = name
        self.instructions = instructions
        self.functions = functions or []
        self.state = AgentState.READY
        self._init_client(**kwargs)

    @abstractmethod
    def _init_client(self, **kwargs):
        """Initialize the client - to be implemented by specific agents"""
        pass

    @abstractmethod
    def send_message(self, message: str, is_function_result: bool = False) -> AgentResponse:
        """Send a message to the agent and get a response"""
        pass


class AnthropicAgent(BaseAgent):
    def _init_client(self, **kwargs):
        self.model = kwargs.get("model", "claude-3-5-sonnet-20241022")
        api_key = kwargs.get("api_key")
        if not api_key:
            raise ValueError("api_key is required for AnthropicAgent")
        self.client = Anthropic(api_key=api_key)
        self.agent_memory = []
    
        
    def _get_tool_schemas(self) -> List[Dict]:
        """Convert functions to Anthropic's tool format"""
        return [
            create_function_schema(func, SchemaFormat.ANTHROPIC)
            for func in self.functions
        ]
        
    def send_message(self, messages:Dict) -> AgentResponse:
        # Get Agent Tool Schema
        tools = self._get_tool_schemas() if self.functions else []
        
        # Get response from Claude
        response = self.client.messages.create(
            model=self.model,
            messages=messages,
            max_tokens=8192,
            tools=tools,
        )
        
        # Get the response texts and the function calls seperately
        text_parts = []
        result = AgentResponse()
        result.function_calls = []
        for block in response.content:
            if block.type == "text":
                text_parts.append(block.text)
        
            elif block.type == "tool_use":
                # Check if this exact function call with these parameters has been made before
                func_call = {
                    "name": block.name,
                    "parameters": block.input,
                }
                result.function_calls.append(func_call)

        result.text = " ".join(text_parts) if text_parts else ""
        return result
                
    def get_agent_memory(self):
        pass

In [12]:
def transfer_to_weather(task:str) -> BaseAgent:
    """Transfer control to the weather bot
    Args:
        task (str, optional): Give the weather bot the task. 
    """
    return weather_bot

def transfer_to_orchestrator(task:str) -> BaseAgent:
    """Transfer control back to the ochestrator when assigned tasks are done or need additional supports
    Args:
        task (str, optional): Update the ochestrator what have been done and what errors have been made. 
    """
    return orchestrator

def transfer_to_code(task:str) -> BaseAgent:
    """Transfer control to the code bot
    Args:
        task (str, optional): Give the code bot the task. 
    """
    return code_bot

# Initialize the weather bot
weather_bot = AnthropicAgent(
    api_key = os.getenv('ANTHROPIC_API_KEY'),
    name="WeatherBot",
    instructions="""
    You are the weather bot. Your role is to:
    1. Get weather information for specified locations
    2. After completing the task, return control to orchestrator
    3. DO NOT suggest additional weather checks unless asked by the user
    4. Initiaate multiple functions in the same result
    """,
    functions=[get_weather, transfer_to_orchestrator],
)

# Initialize the code bot
code_bot = AnthropicAgent(
    name="CodeBot",
    instructions="""
    You are the coding assistant. Your role is to:
    1. Help users with coding tasks and questions
    2. Create env (stick to the env name as python_env), install packages, generate and debug code
    3. Initiaate multiple functions in the same result(create_virtualenv, install_package, execute_code, transfer_to_orchestrator)
    4. After completing a task, return control to orchestrator
    5. DO NOT suggest additional coding tasks unless asked by the user
    6. Always transfer control back to the orchestrator using transfer_to_orchestrator
    """,
    api_key = os.getenv('ANTHROPIC_API_KEY'),
    functions=[
        create_virtualenv,
        install_package,
        execute_code,
        transfer_to_orchestrator,
    ],
)

# Initialize the orchestrator
orchestrator = AnthropicAgent(
    api_key = os.getenv('ANTHROPIC_API_KEY'),
    is_orchestrator = True,
    name="OrchestratorBot",
    instructions="""
    You are the orchestrator bot. Your role is to:
    1. Transfer control to other bots based on user input:
        - Analyze the request and determine which bot should handle it
        - Use transfer functions as follows:
            - Example: transfer_to_agentx(task = "Get temperature in New York")
    
    2. When a taks is done by an agent, it will be transfered to you for you to decide which agent to transfer to for the next task
    """,
    functions=[transfer_to_weather, transfer_to_code],
)

In [13]:
# Usage of the next
func_name = 'get_weather'
next((f for f in [get_weather, transfer_to_orchestrator] if f.__name__ == func_name),None)

<function smart_team.agents.agent_functions.get_weather(city: str) -> str>

In [31]:
active_agent = orchestrator
cross_agent_memories = []
while True:
    messages = []
    user_input = input("\nEnter your request (or 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    messages = [{'role': 'user', 'content': f"{user_input} + {active_agent.instructions} + The following is the hitorical conversation history(null if none): {str(cross_agent_memories)}"}]
    if cross_agent_memories and active_agent.is_ochestrator == True:
        messages.append({'role': 'assistant', 'content': f"The following is the hitorical conversation history: {str(cross_agent_memories)}"})

    result = active_agent.send_message(messages)
    print(result.text)
    cross_agent_memories.append({"role": "user", "content": user_input})
    cross_agent_memories.append({"role": "assistant", "content": result.text})
  
    while result.function_calls:
        for func_call in result.function_calls:
            func_name = func_call["name"]
            func_params = func_call["parameters"]
            cross_agent_memories.append({"role": "assistant", "content": f"{active_agent.name} Is Starting Function Call: {func_name}({func_params})"})
            ################# Start Transfer Logic #################
            if func_name.startswith("transfer_to_"):
                print(f"Transferring to {func_name}!!!")
                func = next(
                            (f for f in active_agent.functions if f.__name__ == func_name),
                            None
                        )
                new_agent = func(**func_params)
                active_agent = new_agent
                print(f'New Agent Name:{active_agent.name}')
                cross_agent_memories.append({"role": "assistant", "content": f"{active_agent.name} is Transferring Function Call: {func_name}({func_params})"})
                agent_memory = [mem for mem in cross_agent_memories if mem["content"].startswith(active_agent.name)]
                # New Agent Function Execution After Transfer
                messages = [{'role': 'user', 'content': (str(agent_memory) + active_agent.instructions).strip()}]
                result = active_agent.send_message(messages)
                
                agent_function_mapping = {fun.__name__: fun for fun in active_agent.functions}
                should_break = False
                while active_agent.is_ochestrator is not True and result.function_calls:
                    # Execute the agent functions
                    for func_call in result.function_calls:
                        try:
                            func_name = func_call["name"]
                            # Check for transfer to orchestrator first
                            if func_name == "transfer_to_orchestrator":
                                should_break = True
                                break
                                
                            func_params = func_call["parameters"]
                            func = agent_function_mapping[func_name]
                            func_result = func(**func_call["parameters"])
                            print(f'{active_agent.name} Finished the Function Result:{func_name} Finished. Result: {func_result}')
                            cross_agent_memories.append({"role": "assistant", "content": f"{active_agent.name} Finished the Function Call: {func_name}({func_params} with {func_result})"})
                            
                        except Exception as e:
                            error_msg = {"role": "assistant", "content": f"Error executing {func_name}: {str(e)}"}
                            print(error_msg)
                            cross_agent_memories.append(error_msg)
                    
                    if should_break:
                        break
                        
                    # Only send new message if we're continuing
                    agent_memory = [mem for mem in cross_agent_memories if mem["content"].startswith(active_agent.name)]
                    messages = [{'role': 'user', 'content': (str(agent_memory) + active_agent.instructions).strip()}]
                    print(messages)
                    result = active_agent.send_message(messages)

Since this request involves writing and executing a Python script to fetch stock data using yfinance, I'll transfer this task to the code bot.
Transferring to transfer_to_code!!!
New Agent Name:CodeBot
CodeBot Finished the Function Result:create_virtualenv Finished. Result: Virtual environment 'python_env' already exists.
[{'role': 'user', 'content': '[{\'role\': \'assistant\', \'content\': "CodeBot is Transferring Function Call: transfer_to_code({\'task\': \'Write and execute a Python script to get the latest TSLA stock price using yfinance. Create a virtual environment, install required packages, and run the code to display the current price.\'})"}, {\'role\': \'assistant\', \'content\': "CodeBot Finished the Function Call: create_virtualenv({\'env_name\': \'python_env\'} with Virtual environment \'python_env\' already exists.)"}]\n    You are the coding assistant. Your role is to:\n    1. Help users with coding tasks and questions\n    2. Create env (stick to the env name as python_